In [2]:
#General imports needed for path
import os 
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

In [3]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping

In [4]:
import route_dynamics.route_elevation.base_df as base
from route_dynamics.route_riders import route_riders as ride

In [5]:
#User defines what routes they want to evaluate
rt_list = [22,101,102,143,150,153,154,156,157,158,159,168,169,177,178,179,180,181,182,183,186,187,190,192]

#Import route shapefile
shapefile_name = '../data/rt' + str(rt_list[0]) + '_pts2.shp'

#Import bus stops shapefile
routes_shp = '../data/Transit_Routes_for_King_County_Metro__transitroute_line.shp'

stops_shp = '../data/Transit_Stops_for_King_County_Metro__transitstop_point.shp'

signals_shp = '../data/traffic_signals2.shp'

streets = '../data/Metro_Transportation_Network_(TNET)_in_King_County_for_Car_Mode___trans_network_car_line.shp'

In [6]:
route_df = base.wrapper(shapefile_name, 6, 6)
route_df.head()

,geometry,distance,elevation,speed_limit,grade
0,POINT (-122.38813 47.56295),0.0000,99.398090,11.175682,0.000000
1,POINT (-122.38798 47.56295),10.9728,99.874774,11.175682,0.043442
2,POINT (-122.38784 47.56295),21.9456,100.272354,11.175682,0.036233
3,POINT (-122.38769 47.56295),32.9184,100.594760,11.175682,0.029382
4,POINT (-122.38754 47.56295),43.8912,100.845922,11.175682,0.022889


In [7]:
trip_df = ride.route_ridership('PM', rt_list[0], 14000)
geometry = trip_df.geometry.values

/Users/ericaeggleton/Route_Dynamics/route_dynamics/route_riders/route_riders.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['geometry'].iloc[i] = copy


In [8]:
xy = []
for i in range(len(geometry)):
    dic = mapping(geometry[i][0])
    coords = dic['coordinates']
    xy.append(coords)
    xy_df = pd.DataFrame(columns = ['STOP_ID','coordinates'])
    xy_df['coordinates'] = xy
    
xy_df

stop_coords = xy_df.coordinates.values

In [9]:
len(stop_coords)

88

In [10]:
signals = gpd.read_file(signals_shp)
signals = signals[signals['Route_Num']==str(rt_list[0])]
geometry = signals.geometry.values
xy = []
for i in range(len(geometry)):
    dic = mapping(geometry[i])
    coords = dic['coordinates']
    xy.append(coords)
    xy_df = pd.DataFrame(columns = ['STOP_ID','coordinates'])
    xy_df['coordinates'] = xy
    
xy_df

signal_coords = xy_df.coordinates.values
signal_coords

array([((-122.38721468411738, 47.54487174863671),),
       ((-122.36591223848691, 47.51736351230742),),
       ((-122.36046009360943, 47.528283928977466),),
       ((-122.37139731666116, 47.5174259120546),),
       ((-122.38677779317288, 47.5611069126548),),
       ((-122.37663707152576, 47.52828465064901),),
       ((-122.38677604222957, 47.56291209080216),),
       ((-122.37688399553113, 47.5174880253816),),
       ((-122.36009964550894, 47.52286151985785),),
       ((-122.36589440460489, 47.52102134912755),),
       ((-122.38719564896368, 47.54668253964439),),
       ((-122.38726107814564, 47.54073076076407),),
       ((-122.38682946854995, 47.55932470413577),),
       ((-122.36047870995853, 47.52469382133861),),
       ((-122.36728387958645, 47.51737832557316),),
       ((-122.36581573707791, 47.50822880157785),)], dtype=object)

In [11]:
def find_knn(
    k,
    candidate_pts,
    test_pts,
    weight=None
    ):
    """ Takes list of points as candidate neighbors (route points) and
        returns one for reach test point (stops) which is the nearest
        route point to the stop.
        """

    num_train_pts = len(candidate_pts)
    num_test_pts = len(test_pts)
    # Find the k nearest neighbors.

    # Initialize array to store distances between data points
    distance_array = np.zeros((num_test_pts, num_train_pts))

    # Loop through rows in test data
    for test_idx in range(num_test_pts):
        test_coord = test_pts[test_idx]

        # Loop through rows in training data
        for train_idx in range(num_train_pts):
            train_coord = candidate_pts[train_idx]

            # Calc distance between training_data and data
            dist = euclidean_distance(
                train_coord,
                test_coord
                )

            # Store distance in array
            distance_array[test_idx, train_idx] = dist

    # print('distance_array = ',distance_array)
    # sort rows corresponding to each unclassified data point
    sorting_idicies = np.argsort(distance_array)

    # Keep only first k columns, corresponding to k nearest neighbors
    # for each unclassified data point
    k_nearest_indicies = sorting_idicies[:,:k]
    # print(k_nearest_indicies.shape)
    # print('k_nearest_indicies = ',k_nearest_indicies)
    k_nearest_distances = distance_array[
        np.arange(num_test_pts)[:,None], k_nearest_indicies
        ]

    # Build array to hold class of each training data point
    points_array = np.asarray([candidate_pts]*num_test_pts)
    k_nearest_neighbors = points_array[
        np.arange(num_test_pts)[:,None], k_nearest_indicies
        ]

    return k_nearest_indicies, k_nearest_neighbors


In [12]:
def euclidean_distance(pt_1, pt_2):
    """ A function that returns the Euclidean distance between a row in the
        intput data to be classified.
        """
    # make sure input are np arrays
    pt_1 = np.array(pt_1.coords)
    pt_2 = np.array(pt_2)

    eucl_dist = np.linalg.norm(pt_2 - pt_1)
    return eucl_dist

In [13]:
stop_nn_indicies, stop_coord_nn = find_knn(
1,
route_df.geometry.values,
stop_coords
)


signal_nn_indicies, singal_coord_nn = find_knn(1,route_df.geometry.values, signal_coords)

route_df = route_df.assign(
is_bus_stop = ([False] * len(route_df.index))
)

route_df = route_df.assign(
is_signal = ([False] * len(route_df.index))
)

route_df = route_df.assign(
is_stop = ([False] * len(route_df.index))
)

for i in stop_nn_indicies.ravel():
    route_df.at[i, 'is_bus_stop'] = True
    route_df.at[i, 'is_stop'] = True
    
for i in signal_nn_indicies.ravel()[::2]:
    route_df.at[i, 'is_stop'] = True
    route_df.at[i, 'is_signal'] = True

In [14]:
len(stop_nn_indicies)

88

In [15]:
route_df.head()

,geometry,distance,elevation,speed_limit,grade,is_bus_stop,is_signal,is_stop
0,POINT (-122.38813 47.56295),0.0000,99.398090,11.175682,0.000000,False,False,False
1,POINT (-122.38798 47.56295),10.9728,99.874774,11.175682,0.043442,False,False,False
2,POINT (-122.38784 47.56295),21.9456,100.272354,11.175682,0.036233,False,False,False
3,POINT (-122.38769 47.56295),32.9184,100.594760,11.175682,0.029382,False,False,False
4,POINT (-122.38754 47.56295),43.8912,100.845922,11.175682,0.022889,False,False,False


In [16]:
route_df['is_stop'].value_counts()

False    1526
True       94
Name: is_stop, dtype: int64

In [17]:
route_df['is_bus_stop'].value_counts()

False    1534
True       86
Name: is_bus_stop, dtype: int64

In [18]:
route_df['is_signal'].value_counts()

False    1612
True        8
Name: is_signal, dtype: int64

In [19]:
np.shape(stop_nn_indicies)

(88, 1)

In [21]:
np.shape(stop_nn_indicies[0])

(1,)